# Code resolution
Code from Dr. Bu

In [1]:
import pandas as pd
import numpy as np

c:\Users\sclab\anaconda3\envs\xai\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\sclab\anaconda3\envs\xai\lib\site-packages\numpy\.libs\libopenblas.fb5ae2tyxyh2ijrdkgdgq3xbklktf43h.gfortran-win_amd64.dll
c:\Users\sclab\anaconda3\envs\xai\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Preprocessing


#### Import and handling missing value

In [2]:
# load all data
dataset = pd.read_csv('../../datasets/household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])

C:\Users\sclab\AppData\Local\Temp\ipykernel_7312\4013634317.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  dataset = pd.read_csv('../../datasets/household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])
C:\Users\sclab\AppData\Local\Temp\ipykernel_7312\4013634317.py:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dataset = pd.read_csv('../../datasets/household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])


In [44]:
# fill missing values with a value at the same time one day ago
def fill_missing(values):
    # values = dataset
    one_day = 60 * 24
    for row in range(values.shape[0]):
        for col in range(values.shape[1]):
            if np.isnan(values[row, col]):
                # dataset.at[row, dataset.columns[col]] = values[row - one_day, col]
                values[row, col] = values[row - one_day, col]
                
    return values

 # mark all missing values
dataset.replace('?', np.nan, inplace=True)
# make dataset numeric
dataset = dataset.astype('float32')
# fill missing
values = fill_missing(dataset.values)
# add a column for for the remainder of sub metering
# values = dataset.values
dataset['sub_metering_4'] = (values[:,0] * 1000 / 60) - (values[:,4] + values[:,5] + values[:,6])

In [45]:
values.shape

(2075259, 8)

In [46]:
(values[:,0] * 1000 / 60) - (values[:,4] + values[:,5] + values[:,6]), values[:,:][:,[0]].reshape(1,-1)[0]


(array([52.26667 , 72.333336, 70.566666, ..., 15.633333, 15.566667,
        15.533334], dtype=float32),
 array([4.216, 5.36 , 5.374, ..., 0.938, 0.934, 0.932], dtype=float32))

In [47]:
np.isnan(vals).sum()

0

In [48]:
dataset.loc[:,dataset.columns[0]].isna().sum()

25979

Create new modified dataset

In [49]:
index = dataset.index
new_dataset_dict = {}
cols = dataset.columns
for i in range(values.shape[1]):
    new_dataset_dict[cols[i]] = values[:,:][:,[i]].reshape(1,-1)[0].tolist()
new_dataset_dict['sub_metering_4'] = ((values[:,0] * 1000 / 60) - (values[:,4] + values[:,5] + values[:,6])).tolist()
df = pd.DataFrame(data=new_dataset_dict, index=index)
df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
datetime,,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.839996,18.4,0.0,1.0,17.0,52.266670
2006-12-16 17:25:00,5.360,0.436,233.630005,23.0,0.0,1.0,16.0,72.333336
2006-12-16 17:26:00,5.374,0.498,233.289993,23.0,0.0,2.0,17.0,70.566666
2006-12-16 17:27:00,5.388,0.502,233.740005,23.0,0.0,1.0,17.0,71.800003
2006-12-16 17:28:00,3.666,0.528,235.679993,15.8,0.0,1.0,17.0,43.099998


In [52]:
np.isnan(np.array(new_dataset_dict['Global_active_power'])).sum(), np.isnan(np.array(values[:,:][:,[i]].reshape(1,-1)[0].tolist())).sum()

(0, 0)

In [54]:
df.isna().sum()

Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
sub_metering_4           0
dtype: int64

In [56]:
df.to_csv('../../datasets/code-resolution/household_power_consumption.csv')

#### Time resolution

In [57]:
# resample data per resolution
time_resolution = '5T'
dataset_resampled = dataset.resample(time_resolution).mean()

In [58]:
dataset.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
datetime,,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.839996,18.4,0.0,1.0,17.0,52.266670
2006-12-16 17:25:00,5.360,0.436,233.630005,23.0,0.0,1.0,16.0,72.333336
2006-12-16 17:26:00,5.374,0.498,233.289993,23.0,0.0,2.0,17.0,70.566666
2006-12-16 17:27:00,5.388,0.502,233.740005,23.0,0.0,1.0,17.0,71.800003
2006-12-16 17:28:00,3.666,0.528,235.679993,15.8,0.0,1.0,17.0,43.099998


In [33]:
dataset_resampled.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_4
datetime,,,,,,,,
2006-12-16 17:20:00,4.2160,0.4180,234.839996,18.400000,0.0,1.0,17.000000,52.266670
2006-12-16 17:25:00,4.6616,0.4972,234.272003,19.960001,0.0,1.4,16.799999,59.493336
2006-12-16 17:30:00,3.8360,0.5116,234.204010,16.560001,0.0,1.2,16.799999,45.933334
2006-12-16 17:35:00,4.6684,0.4100,234.212006,20.000000,0.0,1.0,16.799999,60.006664
2006-12-16 17:40:00,3.9176,0.0616,235.889984,16.760000,0.0,0.0,17.000000,48.293335


In [36]:
rules = np.load('../../resources/association_rules.npy', allow_pickle=True)

In [39]:
rules

array([[1, array(['Attention Type 1'], dtype='<U16'), 0.2184, 0.2184,
        1.0],
       [1, array(['Attention Type 2'], dtype='<U16'), 0.558, 0.558, 1.0],
       [1, array(['CAM Type 0'], dtype='<U10'), 0.2314, 0.2314, 1.0],
       ...,
       [7,
        array(['Global Intensity 25-50%', 'CAM Type 3',
               'Global Active Power 25-50%', 'Global Reactive Power 25-50%',
               'Power Pattern 0', 'Climate Control System 50-75%',
               'Voltage 25-50%'], dtype='<U29')                             ,
        0.0634, 0.4561, 3.8503],
       [7,
        array(['Global Intensity 25-50%', 'Power Pattern 1',
               'Global Active Power 25-50%', 'Global Reactive Power 25-50%',
               'CAM Type 4', 'Climate Control System 50-75%', 'Voltage 25-50%'],
              dtype='<U29')                                                     ,
        0.0602, 0.3355, 4.3499],
       [8,
        array(['Global Intensity 25-50%', 'Power Pattern 1',
               'Globa

In [47]:
counter = 0
for rule in rules:
    # if counter == 200:
    #     break
    counter+=1 
    print(rule)

[1 array(['Attention Type 1'], dtype='<U16') 0.2184 0.2184 1.0]
[1 array(['Attention Type 2'], dtype='<U16') 0.558 0.558 1.0]
[1 array(['CAM Type 0'], dtype='<U10') 0.2314 0.2314 1.0]
[1 array(['Climate Control System 50-75%'], dtype='<U29') 0.3652 0.3652
 1.0]
[1 array(['Global Active Power 25-50%'], dtype='<U26') 0.5557 0.5557 1.0]
[1 array(['Global Intensity 25-50%'], dtype='<U23') 0.4711 0.4711 1.0]
[1 array(['Global Reactive Power 25-50%'], dtype='<U28') 0.8839 0.8839 1.0]
[1 array(['Power Pattern 0'], dtype='<U15') 0.2019 0.2019 1.0]
[1 array(['Power Pattern 1'], dtype='<U15') 0.2227 0.2227 1.0]
[1 array(['Power Pattern 2'], dtype='<U15') 0.3097 0.3097 1.0]
[1 array(['Power Pattern 3'], dtype='<U15') 0.2656 0.2656 1.0]
[1 array(['Voltage 25-50%'], dtype='<U14') 0.9543 0.9543 1.0]
[2
 array(['Attention Type 0', 'Climate Control System 50-75%'], dtype='<U29')
 0.0622 0.3753 1.0276]
[2 array(['Attention Type 0', 'Global Active Power 25-50%'], dtype='<U26')
 0.0878 0.5297 0.9532]
[2 